In [1]:
import keras
from keras.layers import LSTM
import numpy as np
from sklearn.model_selection import KFold, LeaveOneOut
import logging

np.random.seed(42)

logger = logging.getLogger()
logging.basicConfig(filename='../../../results/run_one_hmm.log',level=logging.DEBUG)


Using TensorFlow backend.


In [2]:
from kp_data_loader import KPDataLoader

In [3]:
# Get data again just in case I messed up somewhere
loader = KPDataLoader()
for i in range(1,46):
    loader.load_file('ex{0}a.mid.csv'.format(i))
X, Y, m = loader.get_XX_and_YY()

In [4]:
from keras.preprocessing.sequence import pad_sequences
m

673

In [5]:
X_pad = pad_sequences(X, maxlen=m, padding='post')
Y_pad = pad_sequences(Y, maxlen=m, padding='post')
perm = np.random.permutation(len(X_pad))
X_pad = X_pad[perm]
Y_pad = Y_pad[perm]

In [6]:
sample_weights = np.ones((45, m))
for i in xrange(len(X)):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0

In [7]:
from keras.models import Sequential

In [8]:
# model = Sequential()

In [9]:
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers.wrappers import TimeDistributed

In [10]:
from keras.layers import Embedding

In [11]:
# mask = np.zeros(12)
# model.add(Masking(mask_value=mask, input_shape=(m, 12)))
# model.add(LSTM(50, return_sequences=True, dropout_W=0.4, dropout_U=0.4))
# model.add(TimeDistributed(Dense(12, activation="softmax")))

In [12]:
from keras.utils.np_utils import to_categorical

In [13]:
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               sample_weight_mode='temporal')

In [14]:
# n = len(X)
# j = int(n - (float(n) * 0.5))
# X_train, X_test = X_pad[:j, :], X_pad[j:, :]

In [15]:
# Y_train, Y_test = Y_pad[:j, :], Y_pad[j:, :]

In [16]:
# sample_weights_train, sample_weights_test = sample_weights[:j, :], sample_weights[j:, :]

In [ ]:
# # for custom metrics
import numpy as K

# def weighted_accuracy(y_true, y_pred):
#     score_array *= sample_weights
# score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))

def weighted_accuracy(y_true, y_pred):
    # Only for testing
#     score_array = K.equal(K.argmax(y_true, axis=-1),
#                           K.argmax(y_pred, axis=-1))
#     score_array *= weights
#     score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))
#     return K.mean(score_array)
    total = 0.0
    count = 0.0
    for i, y_i in enumerate(y_true):
        for j, y_ij in enumerate(y_i):
            if sum(y_ij) > 0:
                total += y_ij[y_pred[i][j]]
                count += 1
    return total / count

In [ ]:
# hist = model.fit(X_train, Y_train, batch_size=j, nb_epoch=500, sample_weight=sample_weights_train)

def cross_val(X_pad, Y_pad, sample_weights, m, n=10):
    """
    n : n-crossvalidation
    """

    L = len(X_pad)
    kf = KFold(n_splits=n)

    models = []
    scores = []
    hists = []

    c = 0

    for c, (train_indexes, val_indexes) in enumerate(kf.split(X_pad)):

        logger.debug("On Fold " + str(c))

        xx_train = []
        y_train = []
        xx_val = []
        y_val = []
        sample_weights_train = []
        sample_weights_test = []
        
        for i in train_indexes:
            xx_train.append(X_pad[i][:])
            y_train.append(Y_pad[i][:])
            sample_weights_train.append(sample_weights[i][:])
        for j in val_indexes:
            xx_val.append(X_pad[j][:])
            y_val.append(Y_pad[j][:])
            sample_weights_test.append(sample_weights[j][:])
            
        l = len(xx_train)
        
        xx_train = np.asarray(xx_train)
        xx_val = np.asarray(xx_val)
        y_train = np.asarray(y_train)
        y_val = np.asarray(y_val)
        sample_weights_train = np.asarray(sample_weights_train)
        sample_weights_test = np.asarray(sample_weights_test)
        
        model = Sequential()

        mask = np.zeros(12)
        model.add(Masking(mask_value=mask, input_shape=(m, 12)))
        model.add(LSTM(50, return_sequences=True, dropout_W=0.4, dropout_U=0.4))
        model.add(TimeDistributed(Dense(12, activation="softmax")))
        
        model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              sample_weight_mode='temporal')

#         logger.debug(str(len(X_train)) + "," + str(len(Y_train)))
        hist = model.fit(xx_train, y_train, batch_size=l, nb_epoch=500, sample_weight=sample_weights_train)
        
        hists.append(hist)

        logger.debug("Testing ...")
        Y_prediction = model.predict_classes(xx_val, batch_size=5)
        accuracy = weighted_accuracy(y_val, Y_prediction)
        
        logger.debug("Fold " + str(c) + " scored " + str(accuracy))

        models.append(model)
        scores.append(accuracy)

#     max_score = max(scores)

#     print max_score

#     max_index = 0
#     for idx, score in enumerate(scores):
#         if score == max_score:
#             max_index = idx
#             break
    
    return scores, hists

scores, hists = cross_val(X_pad, Y_pad, sample_weights, m, n=10)

On Fold 0
Epoch 1/500
40/40 [==============================] - 3s - loss: 8.8675
Epoch 2/500
40/40 [==============================] - 1s - loss: 8.7339
Epoch 3/500
40/40 [==============================] - 1s - loss: 8.6341
Epoch 4/500
40/40 [==============================] - 2s - loss: 8.6321
Epoch 5/500
40/40 [==============================] - 2s - loss: 8.5944
Epoch 6/500
40/40 [==============================] - 1s - loss: 8.4776
Epoch 7/500
40/40 [==============================] - 2s - loss: 8.3783
Epoch 8/500
40/40 [==============================] - 2s - loss: 8.3941
Epoch 9/500
40/40 [==============================] - 1s - loss: 8.3918
Epoch 10/500
40/40 [==============================] - 2s - loss: 8.3428
Epoch 11/500
40/40 [==============================] - 2s - loss: 8.3235
Epoch 12/500
40/40 [==============================] - 2s - loss: 8.2634
Epoch 13/500
40/40 [==============================] - 2s - loss: 8.1698
Epoch 14/500
40/40 [==============================] - 2s - loss

In [ ]:
mean

In [ ]:
# Y_prediction = to_categorical(Y_prediction)
# weighted_accuracy(Y_test, Y_prediction, sample_weights_test)
weighted_accuracy(Y_test, Y_prediction)

In [ ]:
model.evaluate(X_test, Y_test, batch_size=20, sample_weight=sample_weights_test)

In [ ]:
Y_prediction[0]

In [ ]:
loader = KPDataLoader()
loader.load_file('ex1a.mid.csv')
X, Y, m = loader.get_XX_and_YY()
X_pad = pad_sequences(X, maxlen=673, padding='post')
Y_pad = pad_sequences(Y, maxlen=673, padding='post')
sample_weights = np.ones((1, m))
for i in xrange(len(X)):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0
Y_prediction = model.predict_classes(X_pad, batch_size=1)

In [ ]:
np.argwhere(Y_pad)[:,2]

In [ ]:
Y_prediction[:101]

In [ ]:
for w in model.get_weights():
    if len(w.shape) > 1:
        plt.figure()
        plt.imshow(w)

In [ ]:
count = 0
for y in Y_prediction:
    if sum(y) > 0:
        count += 1
        print sum(y)
print count

In [ ]:
hist.history

In [ ]:
import cPickle
with open('test.pkl', 'w') as f:
    test = 100, hist
    cPickle.dump(test, f)

In [ ]:
with open('test.pkl', 'r') as f:
    test = cPickle.load(f)